In [1]:
import pyterrier as pt
pt.init()
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex
from pyterrier_adaptive import GAR, CorpusGraph

/home/rathee/p-graph/p-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:


dataset = pt.get_dataset('irds:msmarco-passage')
retriever = PisaIndex.from_dataset('msmarco_passage').bm25()
scorer = pt.text.get_text(dataset, 'text') >> MonoT5ReRanker(verbose=False, batch_size=1)
graph = CorpusGraph.from_dataset('msmarco_passage', 'corpusgraph_bm25_k16').to_limit_k(16)
graph_2 = CorpusGraph.load("msmarco-passage.gbm25.1024").to_limit_k(16)

print(graph_2)
print("#"*100)
nbh = graph_2.neighbours(docid='0')



print("Neighbors", nbh, len(nbh))
print(len(graph_2.edges_data), graph_2.edges_data[100])
print(graph_2.weights_data)
print("#"*100)

graph_3 = graph_2.sparsify_randomly(5) 

nbh = graph_3.neighbours(docid='2180710')
print("Neighbors", nbh, len(nbh))
# print(len(graph_3.edges_data), graph_3.edges_data[100])
# print(graph_3.weights_data)




pipeline = retriever >> GAR(scorer, graph.sparsify_randomly(5)) >> pt.text.get_text(dataset, 'text')
search_results = pipeline.search('clustering hypothesis information retrieval')
print(search_results)



# qid                                        query    docno  rank       score  iteration                                               text
#   1  clustering hypothesis information retrieval  2180710     0   -0.017059          0  Cluster analysis or clustering is the task of ...
#   1  clustering hypothesis information retrieval  8430269     1   -0.166563          1  Clustering is the grouping of a particular set...
#   1  clustering hypothesis information retrieval  1091429     2   -0.208345          1  Clustering is a fundamental data analysis meth...
#   1  clustering hypothesis information retrieval  2180711     3   -0.341018          5  Cluster analysis or clustering is the task of ...
#   1  clustering hypothesis information retrieval  6031959     4   -0.367014          5  Cluster analysis or clustering is the task of ...
#  ..                                          ...      ...   ...         ...        ...                                                ...
#                iteration column indicates which GAR batch the document was scored in ^
#                even=initial retrieval   odd=corpus graph    -1=backfilled

In [ ]:
from pyterrier.measures import *
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

print(dataset)
pt.Experiment(
    [retriever, retriever >> scorer, retriever >> GAR(scorer, graph)],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@1000],
    names=['bm25', 'bm25 >> monot5', 'bm25 >> GAR(monot5)']
)

In [1]:
import pyterrier as pt
pt.init()
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex
from pyterrier_adaptive import GAR, CorpusGraph

/home/rathee/quam/p-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [8]:
import pandas as pd
df = pd.DataFrame(
    [
        ["q1", "chemical reactions", "d1", "professor protor poured the chemicals"],
        ["q1", "chemical reactions", "d2", "chemical brothers turned up the beats"],
    ], columns=["qid", "query","docno", "text"])
textscorer = pt.batchretrieve.TextScorer(takes="docs", body_attr="text", wmodel="BM25")
rtr = textscorer.transform(df)

print(rtr)

15:24:45.866 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
  qid docno                                   text  rank     score  \
0  q1    d1  professor protor poured the chemicals     0 -2.321928   
1  q1    d2  chemical brothers turned up the beats     1 -2.321928   

                query  
0  chemical reactions  
1  chemical reactions  


,qid,query,docno,text,docid
0,q1,chemical reactions,d1,professor protor poured the chemicals,1
1,q1,chemical reactions,d2,chemical brothers turned up the beats,2


In [23]:
batch_size = 64

t5c50 = [187.48, 187.80, 187.75, 187.74, 187.77]
t5c100 = [355.21, 355.67, 355.68, 356.09, 356.03]
t5c250 = [877.27, 877.10, 875.66, 874.03, 875.75]
t5c500 =[1748.35, 1748.71,1748.55, 1748.90, 1748.62]

t5c750 = [2606.68,2606.04,2606.90, 2606.15, 2604.43]
t5c1000 = [3481.02, 3479.99, 3481.54, 3478.95, 3476.82]




quamc50 = [3.38, 3.39, 3.38,3.38, 3.39 ]  # 258.81 , 10
quamc100 = [7.47, 7.50, 7.49, 7.55, 7.56]  #  510.66, 30
quamc250 = [16.76, 16.82, 16.75, 16.84, 16.74 ] ## 1263.16, 50                               
quamc500 = [36.02, 36.09, 36.10, 36.09, 36.01]  #2519.44   , 100
quamc750 = [57.65, 56.73, 56.73, 57.77, 57.93] # 3715.49    ,  150
quamc1000 = [96.43, 95.10, 95.89, 95.68, 95.53]  #4921.22  , 300



batch_size = 16

# quamc50 = [4.47, 4.43, 4.48, 4.41, 4.43 ]  # 255.98 , 10
# quamc100 = [9.40, 9.40, 9.43, 9.43, 9.45]  #481.56, 30
# quamc250 = [21.29, 21.24, 21.16, 21.22, 21.04]  # 1178.04, 50
# quamc500 = [49.65,49.74, 49.71, 49.70, 49.63 ] # 2360 , 100
# quamc750 = [76.17, 76.71, 77.02,76.06, 76.17 ] # 3525.37, 150
# quamc1000 = [117.42, 120.70, 117.08 , 117.31, 117.05]  # 4704.588, 300


In [25]:
import numpy as np

t5std = np.std(quamc1000)
mean = np.mean(quamc1000)
print(mean, t5std)

95.726 0.43701716213439856


In [26]:
batch_size = 64


garc50 = [2.64, 2.64, 2.64, 2.66, 2.63]
garc100 = [5.56, 5.54, 5.57, 5.54, 5.51]
garc250 = [19.32, 19.62, 19.62, 19.57, 19.62]
garc500 = [44.34, 44.26, 44.60, 44.63, 44.45]
garc750 = [69.76, 69.65, 69.83, 69.81, 69.84]
garc1000 = [97.78, 96.48, 96.87, 97.42, 96.72]



batch_size = 16





In [33]:
import numpy as np

t5std = np.std(garc1000)
mean = np.mean(garc1000)
print(mean, t5std)

97.054 0.4766382275898562


In [8]:
a = set([1,2,3])

b = {1: "a", 2: "b", 3: "c"}

a.intersection(b)

{1, 2, 3}